<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Grid-Search" data-toc-modified-id="Grid-Search-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Grid Search</a></span></li><li><span><a href="#Best-params-result" data-toc-modified-id="Best-params-result-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Best params result</a></span></li></ul></div>

In [3]:
nb_name = 'Classification_svm_nb_baseline-augmentation'

In [4]:
from classification_utils import *
from clustering_utils import *
from eda_utils import *
from nn_utils_keras import *
from feature_engineering_utils import *
from data_utils import *
import warnings 
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [5]:
train, test = load_data()
# train, upsampling_info = upsampling_train(train)

train_text, train_label = train_augmentation(train, select_comb=[['text'], ['reply', 'reference_one']])
test_text, test_label = test['text'], test['label']

# test_text = test_text.apply(lambda x: normal_string(x))
# train_text = train_text.apply(lambda x: normal_string(x))


may use cols: 
 ['global_index', 'doc_path', 'label', 'reply', 'reference_one', 'reference_two', 'tag_reply', 'tag_reference_one', 'tag_reference_two', 'Subject', 'From', 'Lines', 'Organization', 'contained_emails', 'long_string', 'text', 'error_message']
combination 1 train:  ['text']
combination 2 train:  ['reply', 'reference_one']


In [6]:
####################################
### label mapper
####################################
labels = sorted(train_label.unique())
label_mapper = dict(zip(labels, range(len(labels))))
train_label = train_label.map(label_mapper)
test_label = test_label.map(label_mapper)
y_train = train_label
y_test = test_label

print(train_text.shape)
print(test_text.shape)
print(train_label.shape)
print(test_label.shape)

(22166,)
(7761,)
(22166,)
(7761,)


In [7]:
set(train_label.tolist())

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}

# Grid Search

In [53]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])
parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
              'clf__alpha': [0.2, 0.4, 0.6, 0.8, 1]}
gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=4)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"NB_cv_result_{nb_name}.xlsx")

clf__alpha:	0.2
tfidf__binary:	True
tfidf__min_df:	3
tfidf__stop_words:	english
tfidf__use_idf:	True
best f1 score: 0.939


In [41]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])
parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
             'clf__penalty':['l2'], 'clf__C':[1,2,3]}
gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=4)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"SVC_cv_result_{nb_name}.xlsx")

clf__C:	3
clf__penalty:	l2
tfidf__binary:	True
tfidf__min_df:	1
tfidf__stop_words:	None
tfidf__use_idf:	True
best f1 score: 0.992


# Best params result

In [18]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, stop_words=None, binary=True, min_df=3)
# tfidf_vectorizer(train_text, test_text, min_df=2, max_df=100)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=500)
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)

num of words: 49023


MemoryError: Unable to allocate 8.10 GiB for an array with shape (22166, 49023) and data type float64

In [15]:
# clf = LinearSVC(penalty="l2", multi_class='ovr', C=3.0, dual=True,)
# clf.fit(X_train, y_train)
pred = clf.predict(X_test)
classification_report = evaluation_report(y_test,  pred, labels=labels)
# roc_auc(y_test, pred)


##########################################
## CV shows the stable result
##########################################
# cv_metrics = ["precision_macro","accuracy", "f1_macro"]
# cv = cross_validate(clf, X_train, y_train,scoring=cv_metrics, cv=4, return_train_score=True)
# cv = pd.DataFrame(cv)
# f1 = cv['test_f1_macro'].mean()
# print("cv average f1 macro: ", f1)

# cv

classification_report:
f1:  {'micro avg': 0.847313490529571, 'macro avg': 0.8349258727936671} 

                          precision  recall  f1-score  support
alt.atheism                   0.812   0.774     0.793    319.0
comp.graphics                 0.757   0.810     0.783    389.0
comp.os.ms-windows.misc       0.770   0.731     0.750    394.0
comp.sys.ibm.pc.hardware      0.743   0.753     0.748    392.0
comp.sys.mac.hardware         0.829   0.857     0.843    385.0
comp.windows.x                0.880   0.815     0.846    395.0
misc.forsale                  0.857   0.908     0.882    390.0
rec.autos                     0.907   0.916     0.912    395.0
rec.motorcycles               0.952   0.957     0.955    398.0
rec.sport.baseball            0.822   0.965     0.888    397.0
rec.sport.hockey              0.992   0.889     0.938    827.0
sci.crypt                     0.934   0.922     0.928    396.0
sci.electronics               0.797   0.738     0.766    393.0
sci.med               

In [17]:
clf = MultinomialNB(alpha=0.2)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
classification_report = evaluation_report(y_test,  pred, labels=labels)

##########################################
## CV shows the stable result
##########################################
# cv_metrics = ["precision_macro","accuracy", "f1_macro"]
# cv = cross_validate(clf, X_train, y_train,scoring=cv_metrics, cv=4, return_train_score=True)
# cv = pd.DataFrame(cv)
# f1 = cv['test_f1_macro'].mean()
# print("cv average f1 macro: ", f1)

# cv

classification_report:
f1:  {'micro avg': 0.7357299317098313, 'macro avg': 0.7360338564743513} 

                          precision  recall  f1-score  support
alt.atheism                   0.832   0.712     0.767    319.0
comp.graphics                 0.747   0.697     0.721    389.0
comp.os.ms-windows.misc       0.795   0.612     0.692    394.0
comp.sys.ibm.pc.hardware      0.654   0.801     0.720    392.0
comp.sys.mac.hardware         0.863   0.821     0.842    385.0
comp.windows.x                0.841   0.828     0.834    395.0
misc.forsale                  0.883   0.795     0.837    390.0
rec.autos                     0.909   0.911     0.910    395.0
rec.motorcycles               0.938   0.955     0.946    398.0
rec.sport.baseball            0.367   0.957     0.531    397.0
rec.sport.hockey              1.000   0.080     0.148    827.0
sci.crypt                     0.786   0.944     0.858    396.0
sci.electronics               0.833   0.700     0.761    393.0
sci.med              